### Imports

In [238]:
keras.backend.clear_session()

In [239]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, Activation
from keras.optimizers import SGD
from keras.initializers import RandomNormal
from keras.models import load_model

### Data Preparation

In [240]:
files = glob.glob("./dataset/fire-images/*.*")
ls_fire = []
for i in files:
    ls_fire.append(['fire-images', i.split("/")[3], '1'])

In [241]:
df_fire = pd.DataFrame(ls_fire, columns=['folder', 'filename', 'label'])
df_fire.head()

,folder,filename,label
0,fire-images,92983122.jpg,1
1,fire-images,wildfire-coverage-521x308.jpg,1
2,fire-images,201307-restroom-fire-safety-fire1.jpg,1
3,fire-images,X-Files_Office_on_fire.jpg,1
4,fire-images,tree-fire.jpg,1


In [242]:
files = glob.glob("./dataset/normal-images-?/*.*")
ls_normal = []
for i in files:
    folder, filename = i.split("/")[2], i.split("/")[3]
    ls_normal.append([f'{folder}', filename, '0'])

In [243]:
df_normal = pd.DataFrame(ls_normal, columns=['folder', 'filename', 'label'])
df_normal.head()

,folder,filename,label
0,normal-images-5,rooms-hotel-tbilisi-suite-bed-and-bathtub-city...,0
1,normal-images-5,unnamed.jpg,0
2,normal-images-5,Tromso.-Photo-credits-Bard-Loken-Innovatioin-N...,0
3,normal-images-5,tumblr_inline_nj85l1oMaC1t5js1c.jpg,0
4,normal-images-5,visa_requirements_for_norway-56e03f4b96539.jpg,0


In [244]:
df = df_fire.append(df_normal, ignore_index=True)
X = df[["folder", "filename"]]
y = df["label"]

In [245]:
df.to_csv("./dataset/fires.csv")

In [246]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['label']), df['label'],
    test_size=0.25, random_state=0
)

X_train['label'] = pd.Series((int(y) for y in y_train), index=X_train.index)
X_test['label'] = pd.Series((int(y) for y in y_test), index=X_test.index)

print("X train\t", X_train.shape, "\tX test", X_test.shape)
print("y train\t", y_train.shape, "\ty test", y_test.shape)

X train	 (488, 3) 	X test (163, 3)
y train	 (488,) 	y test (163,)


In [247]:
# hyper-parameters initialization

img_size = 1024
batch_size = 10
epochs = 10

#### Milad's `fit_generator`

In [248]:
from PIL import Image
from keras.preprocessing.image import img_to_array

# Create data generator for training
def data_gen(df, batch_size):
    while True:
        x_batch = np.zeros((batch_size, 1024, 1024, 3))
        y_batch = np.zeros((batch_size, 1))
        for j in range(len(df)//batch_size):
            b = 0
            for m, k in zip(df['filename'].values[j*batch_size:(j+1)*batch_size], df['label'].values[j*batch_size:(j+1)*batch_size]):
                # print(k, 'dataset/{}/{}'.format(df[df['filename'] == m]['folder'].values[0], m))
                img = Image.open('dataset/{}/{}'.format(df[df['filename'] == m]['folder'].values[0], m)).convert('RGB')
                image_red = img.resize((img_size, img_size))
                x_batch[b] = img_to_array(image_red)
                y_batch[b] = k
                b += 1
            yield (x_batch, y_batch)

#### Modified `fit_generator`

In [233]:
from PIL import Image
from keras.preprocessing.image import img_to_array

def data_gen_2(X, y, batch_size):
    print(X)
    while True:
    # for i in range(5):
        x_batch = np.zeros((batch_size, img_size, img_size, 3))
        y_batch = np.zeros((batch_size, 1))
        for j in range(len(X)//batch_size):
            b = 0
            for m, k in zip(range(len(X[j*batch_size:(j+1)*batch_size])), y[j*batch_size:(j+1)*batch_size]):
                # print(k, 'dataset/{}/{}'.format(X["folder"].iloc[m], X["filename"].iloc[m]))
                img = Image.open('dataset/{}/{}'.format(X["folder"].iloc[m], X["filename"].iloc[m])).convert("RGB")
                image_red = img.resize((img_size, img_size))
                x_batch[b] = img_to_array(image_red)
                y_batch[b] = k
                b += 1
            yield (x_batch, y_batch)

### CNN

#### Baseline model

In [249]:
baseline = Sequential()

baseline.add(Conv2D(
    32, kernel_size=(3,3), 
    input_shape=(img_size, img_size, 3), 
    kernel_initializer='he_uniform', 
    activation="relu")
)
# baseline.add(Conv2D(32, kernel_size=(3,3), input_shape=(1024, 1024, 3), use_bias=False))
# model.add(BatchNormalization())
# baseline.add(Activation("relu"))
baseline.add(MaxPooling2D(pool_size=(2,2)))
# baseline.add(Dropout(0.5))
baseline.add(Flatten())
# baseline.add(Dense(32, activation='relu'))
baseline.add(Dense(1, activation='sigmoid'))

#### working w/ Milad's `fit_generator`

In [250]:
baseline.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=["accuracy"]
)

baseline.fit_generator(
    generator=data_gen(X_train, batch_size=batch_size), 
    steps_per_epoch=len(X_train)//batch_size, 
    epochs=epochs,
    verbose=1
)

Epoch 1/10
48/48 [==============================] - 20s 410ms/step - loss: 40556.1708 - accuracy: 0.7854
Epoch 2/10
48/48 [==============================] - 19s 405ms/step - loss: 7115.3428 - accuracy: 0.8687
Epoch 3/10
48/48 [==============================] - 19s 405ms/step - loss: 2600.1368 - accuracy: 0.8875
Epoch 4/10
48/48 [==============================] - 20s 410ms/step - loss: 2598.4582 - accuracy: 0.9250
Epoch 5/10
48/48 [==============================] - 19s 405ms/step - loss: 1984.6352 - accuracy: 0.9229
Epoch 6/10
48/48 [==============================] - 19s 405ms/step - loss: 1815.9000 - accuracy: 0.9250
Epoch 7/10
48/48 [==============================] - 19s 405ms/step - loss: 5888.0374 - accuracy: 0.8917
Epoch 8/10
48/48 [==============================] - 20s 407ms/step - loss: 701.7731 - accuracy: 0.9646
Epoch 9/10
48/48 [==============================] - 19s 404ms/step - loss: 411.2948 - accuracy: 0.9833
Epoch 10/10
48/48 [==============================] - 19s 405ms/st

In [252]:
loss, accuracy = baseline.evaluate_generator(
    generator=data_gen(X_test, batch_size), 
    steps=len(X_test)//batch_size, verbose=0
)
print("Accuracy = {:.5f}".format(accuracy))

Accuracy = 0.86875


#### not working w/ modified `fit_generator`

In [ ]:
baseline.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=["accuracy"]
)

baseline.fit_generator(
    generator=data_gen_2(X_train, y_train, batch_size=batch_size), 
    steps_per_epoch=len(X_train)//batch_size, 
    epochs=epochs,
    verbose=1
)

In [59]:
loss, accuracy = baseline.evaluate_generator(
    generator=data_gen(X_test, y_test, batch_size), 
    steps=len(X_test)//batch_size, verbose=0
)
print("Accuracy = {:.5f}".format(accuracy))

Accuracy = 0.73750


#### Model 

In [60]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), input_shape=(1024, 1024, 3), activation="relu"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(32, activation="relu", kernel_size=(3,3)))
model.add(MaxPooling2D(pool_size=(3,3)))

# model.add(Conv2D(32, activation="relu", kernel_size=(3,3)))
# model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Dropout(0.2))
model.add(Flatten())

model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=["accuracy"]
)

model.fit_generator(
    generator=data_gen(X_train, y_train, batch_size=batch_size), 
    steps_per_epoch=len(X_train)//batch_size, 
    epochs=epochs,
    verbose=1
)

In [62]:
loss, accuracy = model.evaluate_generator(
    generator=data_gen(X_test, y_test, batch_size), 
    steps=len(X_test)//batch_size, verbose=0
)
print("Accuracy = {:.5f}".format(accuracy))

Accuracy = 0.81250
